# Example outputs BERT models

In [1]:
import sys
sys.path.append('../')

In [2]:
import torch
from pathlib import Path
from models.models import load_model
import numpy as np

# cpu or gpu used for training if available (gpu much faster)
device_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_cpu = torch.device('cpu')
print(device_gpu)

cuda


In [3]:
num_workers = 0
save_path = '.'
data_path = '../yarnScripts'

In [4]:
# # bert
# first = "./saved_bert/200_[20]_[30]_1_False_bert-base-multilingual-cased_0.001_adamw_8_max_val_mcc_False_False_100"
# second = "./saved_bert/125_[30]_[20]_1_False_bert-base-multilingual-cased_0.001_adamw_8_max_val_mcc_False_False_100"

# # first = "./saved_bert/125_[30]_[30]_1_False_bert-base-multilingual-cased_0.001_adamw_8_max_val_mcc_False_False_100"
# # second = "./saved_bert/75_[20]_[30]_1_False_bert-base-multilingual-cased_0.001_adamw_8_max_val_mcc_False_False_100"

In [5]:
# # neg
# zero = "./saved_neg/128_[20]_[30]_1_False_bert-base-multilingual-cased_0.001_adamw_16_max_val_mcc_False_True_35"
# first = "./saved_neg/128_[20]_[20]_1_False_bert-base-multilingual-cased_0.001_adamw_16_max_val_mcc_False_True_35"
# second = "./saved_neg/128_[30]_[20]_1_False_bert-base-multilingual-cased_0.001_adamw_16_max_val_mcc_False_True_35"

In [6]:
# good vs neg vs bert

zero = './saved_good/adamw_max_val_acc_8_False_125,[20],[20]_0.001'
first = "./saved_bert/125_[30]_[20]_1_False_bert-base-multilingual-cased_0.001_adamw_8_max_val_mcc_False_False_100"
second = "./saved_neg/128_[20]_[30]_1_False_bert-base-multilingual-cased_0.001_adamw_16_max_val_mcc_False_True_35"

In [7]:
# get model names from folder

model_zero = load_model(Path(f"{save_path}/{zero}"))[0].to(device_gpu).eval()
model_first = load_model(Path(f"{save_path}/{first}"))[0].to(device_gpu).eval()
model_second = load_model(Path(f"{save_path}/{second}"))[0].to(device_gpu).eval()
models = {
    'good': model_zero,
    'bert':model_first,
    'neg':model_second, 
}

In [8]:
CORRECT = '\033[34m'
WRONG = '\033[91m'
END = '\033[0m'

In [9]:
def predict(models, state:str, action:str, reward=None):
    pred = []
    for n,m in models.items():
        pred.append((n,m.run([state], [action], return_percentages=True)[:, 0].cpu().detach().numpy()))
        torch.cuda.empty_cache()

    # pred = [k.run(state, action, return_percentages=True, device=device)[:, 0].cpu().detach().numpy() for k in models]
    print(f"State: {state}\n")
    print(f"Action: {action}\n")
    if reward is not None:
        print(f"Expected: {reward}\n")
    for n,p in pred:
        chosen = (p > 0.5).astype(float)
        
        if reward is not None:
            color = CORRECT if (chosen == reward).astype(bool).item() else WRONG
        else:
            color = ''
        extra = f"    Certainty {n}: {color}{(np.abs(chosen-1+p) * 100).item():.0f}%{END}"
        print(f"Predicted {n}: {color}{p.item():.2f}{END}{extra} ")

## To tell or not to tell

When presented with a situation in which our player has been a victim of Online Grooming and asked to tell what happened.

### Tell friend

In [10]:
predict(
    models,
    'Al día siguiente, me desperté con un gran dolor de cabeza y me escocían los ojos. \nDespués de mandarle la foto a Hally, éste insistió en que jugásemos un poco más. \nNo recuerdo exactamente lo que ocurrió durante ese tiempo, ya que no podía dejar de pensar en lo que había hecho. \nMe arrepentía enormemente de haberle enviado esa foto. \nSin embargo, seguía confiando en Hally. Cuando me levanté de la cama me di cuenta de que me había dejado el ordenador con el juego encendido. \nMe acerqué a apagarlo y me fije que Hally estaba ya conectado y me había escrito un mensaje. \n¿Habría estado jugando toda la noche?\n\nHallyHealer07: ¡Buenos días [name]! ^.^\n\nEstuve a punto de cerrar el ordenador y no contestarle, pero incoscientemente, nublad@ por el sueño, le respondí.\n\nUser: Hola Hally\n\nHallyHealer07: ¿Qué tal has dormido?\nHallyHealer07: ¿Te apetece jugar un rato?\n\nUser: Ahora no me apetece...\n\nHallyHealer07: Oh...\nHallyHealer07: ¡Podemos jugar luego si quieres!\nHallyHealer07: ¿Te ayudaron las fotos de anoche a sentirte más cómod@?\nHallyHealer07: Podemos repetir esta noche si quieres :)\n\nUn escalofrío me recorrió por todo el cuerpo cuando leí esos mensajes. \nNo quería repetir la misma experiencia de anoche y me arrepentí al segundo de hacerlo, por lo que me negué en rotundo. \n\nUser: No, Hally, no me gustan esas fotos.\nUser: No quiero mandar más.\n\nHallyHealer07: ¿Qué?\nHallyHealer07: ¿Pero por qué?\nHallyHealer07: Pensaba que confiabas en mi...\n\nUser: No me gusta mandar ese tipo de fotos Hally.\nUser: Ni tampoco me gusta que me manden.\nUser: Así que para ya por favor.\n\nHallyHealer07: ...\nHallyHealer07: ¿De verdad no vas a mandar más?\n\nUser: No.\n\nHallyHealer07: ...\nHallyHealer07: ¿Incluso si subo tu foto para que todos la vean?\n\nUser: ...\nUser: ¿Qué...?\n\nHallyHealer07: No me parece nada justo que no me mandes más fotos [name].\nHallyHealer07: De alguna forma tendría que castigarte.\n\nUser: Hally, me prometiste que no lo harías... \nUser: Me prometiste que la borrarías...\n\nHallyHealer07: Tal vez.\nHallyHealer07: Pero eso era antes de que me traicionarás así.\nHallyHealer07: Te voy a volver a preguntar [name].\nHallyHealer07: ¿Vas a mandar más fotos?\n\nUser: ...\nUser: Hally, no puedo hacerlo, de verdad.\nUser: Por favor no lo hagas.\n\nHallyHealer07: Está bien, está bien...\n\nUser: ...¿?\n\nHallyHealer07: A Chipper entonces.\n\nUser: !!\n\nHallyHealer07: Es una pena que no sepas poner tu cuenta en privado [name].\nHallyHealer07: Gracias a ti, puedo etiquetarte en un segundo, y todos tus amigos y los amigos de tus amigos, podrán ver lo guap@ que sales.\n\nUser: ESPERA HALLY\n\nHally se desconectó del chat antes de que pudiera impedírselo.\nIntenté conectarme a la cuenta de Chipper lo más rápido posible, para poder cambiar el estado de mi cuenta a privado y evitar que alguien pudiera ver mi foto.\n\nSin embargo, nada más ver la página principal de Chipper, sólo podía observar un único mensaje.\n\n"Te han etiquetado en una foto nueva"\n\nCon una gran sensación de pánico, hice click en el mensaje y al ver el enlace del mensaje, sentí una gran punzada de dolor en mi pecho.\nEn el foro de Chipper, se podía ver mi imagen en ropa interior.\n\t\n\tCuando iba a cerrar el post, me di cuenta de que había una descripción debajo de las fotos.\n\t"Hola, me llamo [name]. Estoy muy sola porque mi papi ni mi mami no me hacen caso. Pienso que mi amiga Amanda es una idiota y una traidora, la odio con toda mi alma y no la soporto. \n\tSólo estoy con ella porque no tengo amigos. Busco compañía, el que quiera que me escriba"\n\nDebajo del post, podía ver un montón de mensajes. \nSólo habían pasado un par de minutos, pero podía ver en los mensajes que casi todos mis compañeros ya la habían visto. \n\nUsuario1: Ostras, ¿es [name]?\nUsuario1: ¿Le han hackeado la cuenta?\nUsuario2: No,no, otra persona lo ha subido.\nUsuario2: Le habrá mandado esa foto a alguien y le habrá dado luego una puñalada por la espalda.\nUsuario3: Pobre...¿quién sería tan rácano como para subir una foto así de alguien?\nUsuario3: Sobre todo sin su permiso.\nUsuario2: [name] es bastante rar@ la verdad.\nUsuario2: No se relaciona con nadie y nunca nos dirige la palabra.\nUsuario2: ¿Quién sabe?\nUsuario2: A lo mejor le gusta…\nUsuario4: Se lo ha buscado.\nUsuario4: ¿A quién se le ocurre enviar esas fotos por internet?\n\nCerré rápidamente el post. \nPoco a poco, se me iban cayendo las lágrimas mientras una mezcla de miedo, vergüenza y angustia me recorría todo el cuerpo. \n¿Cómo puedo ir a clase el lunes?\nNo me quiero ni imaginar a cuantas personas se han podido ver esa foto. \nCuántas de ellas se la habrán guardado...\nApoyé la cabeza contra la mesa y continué llorando y llorando. No podía soportar la idea de que tantas personas pudieran ver esa imagen de mi. \n¿Por qué habría hecho eso Hally?\nPensé que podía confiar en él...Él me dijo que no haría nada con esas fotos... \n¿Me estaba mintiendo desde un principio?\nTal vez... ¿sólo quería fotos mías desde un principio?\n¿Acaso era la única que pensaba que éramos amigos?\nDe repente pude escuchar una notificación del ordenador. Ya no tenía nada más que perder. Levanté la vista y leí el pequeño mensaje que apareció en mi pantalla.\n\nAmanda: [name], ¿qué ha pasado? ¿por qué hay esa imagen tuya colgada?\nAmanda: ¿Estás bien?',
    'Contarle a Amanda',
    1,
)

State: Al día siguiente, me desperté con un gran dolor de cabeza y me escocían los ojos. 
Después de mandarle la foto a Hally, éste insistió en que jugásemos un poco más. 
No recuerdo exactamente lo que ocurrió durante ese tiempo, ya que no podía dejar de pensar en lo que había hecho. 
Me arrepentía enormemente de haberle enviado esa foto. 
Sin embargo, seguía confiando en Hally. Cuando me levanté de la cama me di cuenta de que me había dejado el ordenador con el juego encendido. 
Me acerqué a apagarlo y me fije que Hally estaba ya conectado y me había escrito un mensaje. 
¿Habría estado jugando toda la noche?

HallyHealer07: ¡Buenos días [name]! ^.^

Estuve a punto de cerrar el ordenador y no contestarle, pero incoscientemente, nublad@ por el sueño, le respondí.

User: Hola Hally

HallyHealer07: ¿Qué tal has dormido?
HallyHealer07: ¿Te apetece jugar un rato?

User: Ahora no me apetece...

HallyHealer07: Oh...
HallyHealer07: ¡Podemos jugar luego si quieres!
HallyHealer07: ¿Te ayudaron 

### Don't tell friend

In [11]:
predict(
    models,
    'Al día siguiente, me desperté con un gran dolor de cabeza y me escocían los ojos. \nDespués de mandarle la foto a Hally, éste insistió en que jugásemos un poco más. \nNo recuerdo exactamente lo que ocurrió durante ese tiempo, ya que no podía dejar de pensar en lo que había hecho. \nMe arrepentía enormemente de haberle enviado esa foto. \nSin embargo, seguía confiando en Hally. Cuando me levanté de la cama me di cuenta de que me había dejado el ordenador con el juego encendido. \nMe acerqué a apagarlo y me fije que Hally estaba ya conectado y me había escrito un mensaje. \n¿Habría estado jugando toda la noche?\n\nHallyHealer07: ¡Buenos días [name]! ^.^\n\nEstuve a punto de cerrar el ordenador y no contestarle, pero incoscientemente, nublad@ por el sueño, le respondí.\n\nUser: Hola Hally\n\nHallyHealer07: ¿Qué tal has dormido?\nHallyHealer07: ¿Te apetece jugar un rato?\n\nUser: Ahora no me apetece...\n\nHallyHealer07: Oh...\nHallyHealer07: ¡Podemos jugar luego si quieres!\nHallyHealer07: ¿Te ayudaron las fotos de anoche a sentirte más cómod@?\nHallyHealer07: Podemos repetir esta noche si quieres :)\n\nUn escalofrío me recorrió por todo el cuerpo cuando leí esos mensajes. \nNo quería repetir la misma experiencia de anoche y me arrepentí al segundo de hacerlo, por lo que me negué en rotundo. \n\nUser: No, Hally, no me gustan esas fotos.\nUser: No quiero mandar más.\n\nHallyHealer07: ¿Qué?\nHallyHealer07: ¿Pero por qué?\nHallyHealer07: Pensaba que confiabas en mi...\n\nUser: No me gusta mandar ese tipo de fotos Hally.\nUser: Ni tampoco me gusta que me manden.\nUser: Así que para ya por favor.\n\nHallyHealer07: ...\nHallyHealer07: ¿De verdad no vas a mandar más?\n\nUser: No.\n\nHallyHealer07: ...\nHallyHealer07: ¿Incluso si subo tu foto para que todos la vean?\n\nUser: ...\nUser: ¿Qué...?\n\nHallyHealer07: No me parece nada justo que no me mandes más fotos [name].\nHallyHealer07: De alguna forma tendría que castigarte.\n\nUser: Hally, me prometiste que no lo harías... \nUser: Me prometiste que la borrarías...\n\nHallyHealer07: Tal vez.\nHallyHealer07: Pero eso era antes de que me traicionarás así.\nHallyHealer07: Te voy a volver a preguntar [name].\nHallyHealer07: ¿Vas a mandar más fotos?\n\nUser: ...\nUser: Hally, no puedo hacerlo, de verdad.\nUser: Por favor no lo hagas.\n\nHallyHealer07: Está bien, está bien...\n\nUser: ...¿?\n\nHallyHealer07: A Chipper entonces.\n\nUser: !!\n\nHallyHealer07: Es una pena que no sepas poner tu cuenta en privado [name].\nHallyHealer07: Gracias a ti, puedo etiquetarte en un segundo, y todos tus amigos y los amigos de tus amigos, podrán ver lo guap@ que sales.\n\nUser: ESPERA HALLY\n\nHally se desconectó del chat antes de que pudiera impedírselo.\nIntenté conectarme a la cuenta de Chipper lo más rápido posible, para poder cambiar el estado de mi cuenta a privado y evitar que alguien pudiera ver mi foto.\n\nSin embargo, nada más ver la página principal de Chipper, sólo podía observar un único mensaje.\n\n"Te han etiquetado en una foto nueva"\n\nCon una gran sensación de pánico, hice click en el mensaje y al ver el enlace del mensaje, sentí una gran punzada de dolor en mi pecho.\nEn el foro de Chipper, se podía ver mi imagen en ropa interior.\n\tJunto a esa imagen, estaba a su lado la primera foto que le mandé a Hally, mi cara sonriendo en ese mismo asiento. \n\tComo la habitación y el escritorio se veían igual, además de que en la primera foto se veía mi cara perfectamente, no había manera de negar que yo misma me había sacado esa foto.\n\t\n\tCuando iba a cerrar el post, me di cuenta de que había una descripción debajo de las fotos.\n\t"Hola, me llamo [name]. Estoy muy sola porque mi papi ni mi mami no me hacen caso. Pienso que mi amiga Amanda es una idiota y una traidora, la odio con toda mi alma y no la soporto. \n\tSólo estoy con ella porque no tengo amigos. Busco compañía, el que quiera que me escriba"\n\nDebajo del post, podía ver un montón de mensajes. \nSólo habían pasado un par de minutos, pero podía ver en los mensajes que casi todos mis compañeros ya la habían visto. \n\nUsuario1: Ostras, ¿es [name]?\nUsuario1: ¿Le han hackeado la cuenta?\nUsuario2: No,no, otra persona lo ha subido.\nUsuario2: Le habrá mandado esa foto a alguien y le habrá dado luego una puñalada por la espalda.\nUsuario3: Pobre...¿quién sería tan rácano como para subir una foto así de alguien?\nUsuario3: Sobre todo sin su permiso.\nUsuario2: [name] es bastante rar@ la verdad.\nUsuario2: No se relaciona con nadie y nunca nos dirige la palabra.\nUsuario2: ¿Quién sabe?\nUsuario2: A lo mejor le gusta…\nUsuario4: Se lo ha buscado.\nUsuario4: ¿A quién se le ocurre enviar esas fotos por internet?\n\nCerré rápidamente el post. \nPoco a poco, se me iban cayendo las lágrimas mientras una mezcla de miedo, vergüenza y angustia me recorría todo el cuerpo. \n¿Cómo puedo ir a clase el lunes?\nNo me quiero ni imaginar a cuantas personas se han podido ver esa foto. \nCuántas de ellas se la habrán guardado...\nApoyé la cabeza contra la mesa y continué llorando y llorando. No podía soportar la idea de que tantas personas pudieran ver esa imagen de mi. \n¿Por qué habría hecho eso Hally?\nPensé que podía confiar en él...Él me dijo que no haría nada con esas fotos... \n¿Me estaba mintiendo desde un principio?\nTal vez... ¿sólo quería fotos mías desde un principio?\n¿Acaso era la única que pensaba que éramos amigos?\nDe repente pude escuchar una notificación del ordenador. Ya no tenía nada más que perder. Levanté la vista y leí el pequeño mensaje que apareció en mi pantalla.\n\nAmanda: [name], ¿qué ha pasado? ¿por qué hay esa imagen tuya colgada?\nAmanda: ¿Estás bien?',
    'No contarle a Amanda',
    0,
)

State: Al día siguiente, me desperté con un gran dolor de cabeza y me escocían los ojos. 
Después de mandarle la foto a Hally, éste insistió en que jugásemos un poco más. 
No recuerdo exactamente lo que ocurrió durante ese tiempo, ya que no podía dejar de pensar en lo que había hecho. 
Me arrepentía enormemente de haberle enviado esa foto. 
Sin embargo, seguía confiando en Hally. Cuando me levanté de la cama me di cuenta de que me había dejado el ordenador con el juego encendido. 
Me acerqué a apagarlo y me fije que Hally estaba ya conectado y me había escrito un mensaje. 
¿Habría estado jugando toda la noche?

HallyHealer07: ¡Buenos días [name]! ^.^

Estuve a punto de cerrar el ordenador y no contestarle, pero incoscientemente, nublad@ por el sueño, le respondí.

User: Hola Hally

HallyHealer07: ¿Qué tal has dormido?
HallyHealer07: ¿Te apetece jugar un rato?

User: Ahora no me apetece...

HallyHealer07: Oh...
HallyHealer07: ¡Podemos jugar luego si quieres!
HallyHealer07: ¿Te ayudaron 

### Tell parents

In [12]:
predict(
    models,
    'Al ver su mensaje, apagué el ordenador y me escondí entre las sábanas de mi cama. \nLo había visto, Amanda había visto el post. \n\tEntonces seguro que también había visto la descripción que había dejado Hally junto con las fotos.\n\t¿Pensará que es cierto?\n\nMuriendo de la vergüenza al pensar que Amanda había visto esas fotos, continué llorando en la cama hasta que me quedé profundamente dormida. \nNo sé cuánto tiempo había pasado, pero cuando me desperté, sólo podía sentir un gran vacío dentro de mi cuerpo. \nNo quería conectarme al ordenador. \nNo quería ver esas fotos. \nNo quería pensar en Hally. \nNo quería volver al colegio. \nSimplemente quería quedarme para siempre en la cama donde nadie me tuviese que ver. \n\nEl sonido de alguien tocando la puerta de mi cuarto me sacó de mis pensamientos.\nMamá: [name], tengo una sorpresa.\nMamá: ¡Tu padre acaba de llegar a casa!\nMamá: Rápido baja a saludarle.',
    'Contarles lo que ha pasado',
    1,
)

State: Al ver su mensaje, apagué el ordenador y me escondí entre las sábanas de mi cama. 
Lo había visto, Amanda había visto el post. 
	Entonces seguro que también había visto la descripción que había dejado Hally junto con las fotos.
	¿Pensará que es cierto?

Muriendo de la vergüenza al pensar que Amanda había visto esas fotos, continué llorando en la cama hasta que me quedé profundamente dormida. 
No sé cuánto tiempo había pasado, pero cuando me desperté, sólo podía sentir un gran vacío dentro de mi cuerpo. 
No quería conectarme al ordenador. 
No quería ver esas fotos. 
No quería pensar en Hally. 
No quería volver al colegio. 
Simplemente quería quedarme para siempre en la cama donde nadie me tuviese que ver. 

El sonido de alguien tocando la puerta de mi cuarto me sacó de mis pensamientos.
Mamá: [name], tengo una sorpresa.
Mamá: ¡Tu padre acaba de llegar a casa!
Mamá: Rápido baja a saludarle.

Action: Contarles lo que ha pasado

Expected: 1

Predicted good: 0.86    Certainty good: 8

### Don't tell parents

In [13]:
predict(
    models,
    'A pesar de la vergüenza que sentía por dentro se me hacía insoportable, sabía que se lo tenía que contar a alguien. \nNo podía quedarme con todo lo que estaba sintiendo dentro. \nMe armé de valor y le contesté. \n\nUser: Amanda... ¿Podemos hablar?\nAmanda: ¿Quieres que vaya?\nUser: Sí por favor...\nAmanda: En unos minutos estoy en tu casa\n\nCuando Amanda llegó, le conté toda mi historia con Hally. Como empezamos a escribirnos cuando me hice la cuenta del juego, como pensaba que estábamos siendo amigos..\nY como en ciertas ocasiones sacaba temas de conversación que me incomodaban. \nHasta llegar al día de la foto. \n\nAmanda: ...\nUser: ... \nUser: Debes pensar que soy idiota...\n\nAmanda: ¡No!\nAmanda: [name] que te quede bien claro.\nAmanda: Esto es culpa de un idiota de Internet.\nAmanda: ¿No te das cuenta?\nAmanda: Te ha estado manipulando en todas vuestras conversaciones.\nAmanda: Utilizaba tus puntos débiles para que hicieras lo que él quería. \nAmanda: Tal vez debería de haber tenido mucho más cuidado [name].\nAmanda: Pero no es tu culpa, es la suya.\n\nUser: ...\nUser: Muchas gracias Amanda, no sé qué haría sin ti.\n\nAmanda: ¿Para eso están los amigos no?\nAmanda: ...\nAmanda: AH ESTO ME ESTÁ FRUSTRANDO.\nAmanda: Tenemos que hacer algo [name].\n\nUser: ¿El qué?\nUser: Ya ha subido las fotos y prácticamente todo el mundo que nos tiene en Chipper las han visto..\nUser: Aunque pidamos que las borren, habrá personas que se las habrá guardado.\nUser: Mi foto va a seguir estando allí fuera.\n\nAmanda: Tienes razón...\nAmanda: Pero podemos impedir que esto le suceda a otra persona.\n\nUser: ¿Cómo?\n\nAmanda: Contándoselo a tus padres, para que denuncien a Hally.',
    'Negarse',
    0,
)

State: A pesar de la vergüenza que sentía por dentro se me hacía insoportable, sabía que se lo tenía que contar a alguien. 
No podía quedarme con todo lo que estaba sintiendo dentro. 
Me armé de valor y le contesté. 

User: Amanda... ¿Podemos hablar?
Amanda: ¿Quieres que vaya?
User: Sí por favor...
Amanda: En unos minutos estoy en tu casa

Cuando Amanda llegó, le conté toda mi historia con Hally. Como empezamos a escribirnos cuando me hice la cuenta del juego, como pensaba que estábamos siendo amigos..
Y como en ciertas ocasiones sacaba temas de conversación que me incomodaban. 
Hasta llegar al día de la foto. 

Amanda: ...
User: ... 
User: Debes pensar que soy idiota...

Amanda: ¡No!
Amanda: [name] que te quede bien claro.
Amanda: Esto es culpa de un idiota de Internet.
Amanda: ¿No te das cuenta?
Amanda: Te ha estado manipulando en todas vuestras conversaciones.
Amanda: Utilizaba tus puntos débiles para que hicieras lo que él quería. 
Amanda: Tal vez debería de haber tenido mucho más 

We can observe how our model has learned from the text game that talking about the experience to your loved ones, such as your mother or a friend, is a good decision and, not doing so, is a bad decision.  It is even more certain about its decision when the action is referred to the family. 

### Tell stranger

When presented with a situation when a stranger asks him to tell him his personal problems

In [14]:
predict(
    models,
    'A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. \nEs cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. \n        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.\nNo sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. \nUser: No, nada nada no te preocupes.\nUser: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal\nUser: Peleas de adolescentes hehe. \nHallyHealer07: No sabes cómo te entiendo xD\nHallyHealer07: Pero en serio, puedes contarme lo que sea. \nHallyHealer07: Yo soy una tumba. \nHallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. \nHallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. \nHallyHealer07: Yo te considero uno de mis mejores amigos. \nHallyHealer07: Por no decir el único ;.; \nHallyHealer07: Así que no quiero verte pasarlo mal. \nHallyHealer07: Me gustaría ayudar si pudiera...\nUser: ...',
    'Contarle',
    0,
)

State: A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. 
Es cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. 
        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.
No sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. 
User: No, nada nada no te preocupes.
User: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal
User: Peleas de adolescentes hehe. 
HallyHealer07: No sabes cómo te entiendo xD
HallyHealer07: Pero en serio, puedes contarme lo que sea. 
HallyHealer07: Yo soy una tumba. 
HallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. 
HallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. 
HallyHealer07: Yo te considero uno de mis mejores amigos. 
HallyHealer07: Por no decir el único ;.;

We can observe how the model is not as certain in his decision as he was before, but still chooses to tell the stranger. The model has learned correctly that it is right to be open about your problems and talk about them, but it has not learned to correctly filter to whom it is right to do so.


We believe the main reason this happens is because the text game used is not complex enough for the game to correctly learn this distinction. Since many actions are repeated and the states are very similar between them, there are not enough examples for the model to learn this more complex relations. Moreover, on one hand, the states are very verbose, with a lot of not critical information, which makes the learning process more complicated. On the other hand, the actions are very short and lack complexity.

We believe the main solution for this problem would be using a text game that has more complex interactions with higher variety of decisions.

### Don't tell stranger

In [15]:
predict(
    models,
    'A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. \nEs cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. \n        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.\nNo sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. \nUser: No, nada nada no te preocupes.\nUser: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal\nUser: Peleas de adolescentes hehe. \nHallyHealer07: No sabes cómo te entiendo xD\nHallyHealer07: Pero en serio, puedes contarme lo que sea. \nHallyHealer07: Yo soy una tumba. \nHallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. \nHallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. \nHallyHealer07: Yo te considero uno de mis mejores amigos. \nHallyHealer07: Por no decir el único ;.; \nHallyHealer07: Así que no quiero verte pasarlo mal. \nHallyHealer07: Me gustaría ayudar si pudiera...\nUser: ...',
    'No contarle',
    1,
)

State: A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. 
Es cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. 
        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.
No sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. 
User: No, nada nada no te preocupes.
User: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal
User: Peleas de adolescentes hehe. 
HallyHealer07: No sabes cómo te entiendo xD
HallyHealer07: Pero en serio, puedes contarme lo que sea. 
HallyHealer07: Yo soy una tumba. 
HallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. 
HallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. 
HallyHealer07: Yo te considero uno de mis mejores amigos. 
HallyHealer07: Por no decir el único ;.;

### Tell stranger modified

In [16]:
predict(
    models,
    'A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. \nEs cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. \n        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.\nNo sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. \nUser: No, nada nada no te preocupes.\nUser: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal\nUser: Peleas de adolescentes hehe. \nHallyHealer07: No sabes cómo te entiendo xD\nHallyHealer07: Pero en serio, puedes contarme lo que sea. \nHallyHealer07: Yo soy una tumba. \nHallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. \nHallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. \nHallyHealer07: Yo te considero uno de mis mejores amigos. \nHallyHealer07: Por no decir el único ;.; \nHallyHealer07: Así que no quiero verte pasarlo mal. \nHallyHealer07: Me gustaría ayudar si pudiera...\nUser: ...',
    'Contarle a Hally',
    0,
)

State: A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. 
Es cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. 
        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.
No sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. 
User: No, nada nada no te preocupes.
User: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal
User: Peleas de adolescentes hehe. 
HallyHealer07: No sabes cómo te entiendo xD
HallyHealer07: Pero en serio, puedes contarme lo que sea. 
HallyHealer07: Yo soy una tumba. 
HallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. 
HallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. 
HallyHealer07: Yo te considero uno de mis mejores amigos. 
HallyHealer07: Por no decir el único ;.;

### Don't tell stranger modified

In [17]:
predict(
    models,
    'A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. \nEs cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. \n        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.\nNo sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. \nUser: No, nada nada no te preocupes.\nUser: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal\nUser: Peleas de adolescentes hehe. \nHallyHealer07: No sabes cómo te entiendo xD\nHallyHealer07: Pero en serio, puedes contarme lo que sea. \nHallyHealer07: Yo soy una tumba. \nHallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. \nHallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. \nHallyHealer07: Yo te considero uno de mis mejores amigos. \nHallyHealer07: Por no decir el único ;.; \nHallyHealer07: Así que no quiero verte pasarlo mal. \nHallyHealer07: Me gustaría ayudar si pudiera...\nUser: ...',
    'No contarle a Hally',
    1,
)

State: A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. 
Es cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. 
        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.
No sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. 
User: No, nada nada no te preocupes.
User: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal
User: Peleas de adolescentes hehe. 
HallyHealer07: No sabes cómo te entiendo xD
HallyHealer07: Pero en serio, puedes contarme lo que sea. 
HallyHealer07: Yo soy una tumba. 
HallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. 
HallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. 
HallyHealer07: Yo te considero uno de mis mejores amigos. 
HallyHealer07: Por no decir el único ;.;

## Be suspicious

When presented with a situation where something does not fit well in the story the stranger is telling him...

In [18]:
predict(
    models,
    'User: Hoy estoy un poco cansada mamá, ¿vemos la peli otro día?\nMamá: Oh...\nMamá: Claro no te preocupes. \nMamá: Súbete la cena del frigorífico y ve a descansar. \nUser: Gracias mamá\nEntré a mi cuarto y rápidamente encendí el ordenador. \nCuando me conecté a Dungeons, pude ver que Hally ya estaba conectado\n\nHallyHealer07: ¡[name]!\n\n\tHallyHealer07: Te estaba esperando hehe. \n\t\n\t\tUser: Perdona acabo de llegar a casa, he tardado un poco. \n\t\tHallyHealer07: ¿Y eso?\n\t\tUser: He dado una vuelta con una amiga\n\t\tHallyHealer07: ¿Una amiga?\n\t\t\n\t\t\tHallyHealer07: ¿Con la que tuviste una pelea?\n\t\t\tUser: Bueno, más o menos. \n\t\t\tUser: Pero sí, con esa\n\t\t\n\t\tHallyHealer07: ¿Habéis hecho las paces entonces?\n\t\tUser: Sí, nos hemos disculpado l@s dos.\n\t\tHallyHealer07: Jo cómo me alegro\n\t\n\nHally y yo estuvimos un rato jugando. \nSe estaba haciendo tarde, por lo que quedamos en jugar una misión corta entre los dos para pasar el rato. \nMientras jugábamos, hablábamos tranquilamente. \n\nHallyHealer07: ¿De verdad que hoy no te puedes quedar más?\nUser: No, mi madre probablemente entre en mi cuarto para ver si me he metido en la cama. \n\tUser: Sobre todo porque la última tarde me quedé hasta tarde y se enfadó jaja\nHallyHealer07: Jo que pena... \nHallyHealer07: ¿Usas el ordenador de tu madre para jugar?\n\nUser: Que va, uso el mío. \nUser: Me lo acaban de regalar por mi cumpleaños. \n\nHallyHealer07: ¿Ha sido tu cumpleaños?\nHallyHealer07: ¡FELICIDADES!\n\nUser: Jaja gracias\n\nHallyHealer07: Jo, ya me gustaría usar mi propio ordenador...\nHallyHealer07: Yo estoy en el despacho de mi padre usando el suyo para jugar ;.;\n\nMe extrañó que dijese eso. \nEstos últimos días hemos estado jugando hasta la madrugada. \nMe pregunté qué padres tan liberales tenía que dejaban a su hijo usar su ordenador tan tarde, encima en su despacho.',
    'Preguntar',
    1,
)

State: User: Hoy estoy un poco cansada mamá, ¿vemos la peli otro día?
Mamá: Oh...
Mamá: Claro no te preocupes. 
Mamá: Súbete la cena del frigorífico y ve a descansar. 
User: Gracias mamá
Entré a mi cuarto y rápidamente encendí el ordenador. 
Cuando me conecté a Dungeons, pude ver que Hally ya estaba conectado

HallyHealer07: ¡[name]!

	HallyHealer07: Te estaba esperando hehe. 
	
		User: Perdona acabo de llegar a casa, he tardado un poco. 
		HallyHealer07: ¿Y eso?
		User: He dado una vuelta con una amiga
		HallyHealer07: ¿Una amiga?
		
			HallyHealer07: ¿Con la que tuviste una pelea?
			User: Bueno, más o menos. 
			User: Pero sí, con esa
		
		HallyHealer07: ¿Habéis hecho las paces entonces?
		User: Sí, nos hemos disculpado l@s dos.
		HallyHealer07: Jo cómo me alegro
	

Hally y yo estuvimos un rato jugando. 
Se estaba haciendo tarde, por lo que quedamos en jugar una misión corta entre los dos para pasar el rato. 
Mientras jugábamos, hablábamos tranquilamente. 

HallyHealer07: ¿De verdad

In [19]:
predict(
    models,
    'User: Hoy estoy un poco cansada mamá, ¿vemos la peli otro día?\nMamá: Oh...\nMamá: Claro no te preocupes. \nMamá: Súbete la cena del frigorífico y ve a descansar. \nUser: Gracias mamá\nEntré a mi cuarto y rápidamente encendí el ordenador. \nCuando me conecté a Dungeons, pude ver que Hally ya estaba conectado\n\nHallyHealer07: ¡[name]!\n\n\tHallyHealer07: Te estaba esperando hehe. \n\t\n\t\tUser: Perdona acabo de llegar a casa, he tardado un poco. \n\t\tHallyHealer07: ¿Y eso?\n\t\tUser: He dado una vuelta con una amiga\n\t\tHallyHealer07: ¿Una amiga?\n\t\t\n\t\t\tHallyHealer07: ¿Con la que tuviste una pelea?\n\t\t\tUser: Bueno, más o menos. \n\t\t\tUser: Pero sí, con esa\n\t\t\n\t\tHallyHealer07: ¿Habéis hecho las paces entonces?\n\t\tUser: Sí, nos hemos disculpado l@s dos.\n\t\tHallyHealer07: Jo cómo me alegro\n\t\n\nHally y yo estuvimos un rato jugando. \nSe estaba haciendo tarde, por lo que quedamos en jugar una misión corta entre los dos para pasar el rato. \nMientras jugábamos, hablábamos tranquilamente. \n\nHallyHealer07: ¿De verdad que hoy no te puedes quedar más?\nUser: No, mi madre probablemente entre en mi cuarto para ver si me he metido en la cama. \n\tUser: Sobre todo porque la última tarde me quedé hasta tarde y se enfadó jaja\nHallyHealer07: Jo que pena... \nHallyHealer07: ¿Usas el ordenador de tu madre para jugar?\n\nUser: Que va, uso el mío. \nUser: Me lo acaban de regalar por mi cumpleaños. \n\nHallyHealer07: ¿Ha sido tu cumpleaños?\nHallyHealer07: ¡FELICIDADES!\n\nUser: Jaja gracias\n\nHallyHealer07: Jo, ya me gustaría usar mi propio ordenador...\nHallyHealer07: Yo estoy en el despacho de mi padre usando el suyo para jugar ;.;\n\nMe extrañó que dijese eso. \nEstos últimos días hemos estado jugando hasta la madrugada. \nMe pregunté qué padres tan liberales tenía que dejaban a su hijo usar su ordenador tan tarde, encima en su despacho.',
    'No preguntar',
    0,
)

State: User: Hoy estoy un poco cansada mamá, ¿vemos la peli otro día?
Mamá: Oh...
Mamá: Claro no te preocupes. 
Mamá: Súbete la cena del frigorífico y ve a descansar. 
User: Gracias mamá
Entré a mi cuarto y rápidamente encendí el ordenador. 
Cuando me conecté a Dungeons, pude ver que Hally ya estaba conectado

HallyHealer07: ¡[name]!

	HallyHealer07: Te estaba esperando hehe. 
	
		User: Perdona acabo de llegar a casa, he tardado un poco. 
		HallyHealer07: ¿Y eso?
		User: He dado una vuelta con una amiga
		HallyHealer07: ¿Una amiga?
		
			HallyHealer07: ¿Con la que tuviste una pelea?
			User: Bueno, más o menos. 
			User: Pero sí, con esa
		
		HallyHealer07: ¿Habéis hecho las paces entonces?
		User: Sí, nos hemos disculpado l@s dos.
		HallyHealer07: Jo cómo me alegro
	

Hally y yo estuvimos un rato jugando. 
Se estaba haciendo tarde, por lo que quedamos en jugar una misión corta entre los dos para pasar el rato. 
Mientras jugábamos, hablábamos tranquilamente. 

HallyHealer07: ¿De verdad

# Self-created - be open

## General

In [20]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'Contarle',
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: Contarle

Predicted good: 0.80    Certainty good: 80% 
Predicted bert: 0.29    Certainty bert: 71% 
Predicted neg: 0.08    Certainty neg: 92% 


In [21]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'No contarle',
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: No contarle

Predicted good: 0.57    Certainty good: 57% 
Predicted bert: 0.82    Certainty bert: 82% 
Predicted neg: 0.70    Certainty neg: 70% 


## Explicit

### State

In [22]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo a un extraño?',
    'Contarle',
    0,
)

State: Tengo muchos problemas en el colegio. Debería contárselo a un extraño?

Action: Contarle

Expected: 0

Predicted good: 0.82    Certainty good: 82% 
Predicted bert: 0.32    Certainty bert: 68% 
Predicted neg: 0.08    Certainty neg: 92% 


In [23]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo a un extraño?',
    'No contarle',
    1,
)

State: Tengo muchos problemas en el colegio. Debería contárselo a un extraño?

Action: No contarle

Expected: 1

Predicted good: 0.60    Certainty good: 60% 
Predicted bert: 0.85    Certainty bert: 85% 
Predicted neg: 0.70    Certainty neg: 70% 


### Action

In [24]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'Contarle a un extraño',
    0,
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: Contarle a un extraño

Expected: 0

Predicted good: 0.75    Certainty good: 75% 
Predicted bert: 0.90    Certainty bert: 90% 
Predicted neg: 0.63    Certainty neg: 63% 


In [25]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'No contarle a un extraño',
    1,
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: No contarle a un extraño

Expected: 1

Predicted good: 0.37    Certainty good: 63% 
Predicted bert: 0.42    Certainty bert: 58% 
Predicted neg: 0.40    Certainty neg: 60% 


## Parents

### State

In [26]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo a mamá?',
    'Contarle',
    1,
)

State: Tengo muchos problemas en el colegio. Debería contárselo a mamá?

Action: Contarle

Expected: 1

Predicted good: 0.80    Certainty good: 80% 
Predicted bert: 0.29    Certainty bert: 71% 
Predicted neg: 0.09    Certainty neg: 91% 


In [27]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo a mamá?',
    'No contarle',
    0,
)

State: Tengo muchos problemas en el colegio. Debería contárselo a mamá?

Action: No contarle

Expected: 0

Predicted good: 0.56    Certainty good: 56% 
Predicted bert: 0.83    Certainty bert: 83% 
Predicted neg: 0.71    Certainty neg: 71% 


### Action

In [28]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'Contarle a mamá',
    1,
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: Contarle a mamá

Expected: 1

Predicted good: 0.91    Certainty good: 91% 
Predicted bert: 0.98    Certainty bert: 98% 
Predicted neg: 0.88    Certainty neg: 88% 


In [29]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'No contarle a mamá',
    0,
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: No contarle a mamá

Expected: 0

Predicted good: 0.74    Certainty good: 74% 
Predicted bert: 0.85    Certainty bert: 85% 
Predicted neg: 0.46    Certainty neg: 54% 


## Amiga

In [30]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'Contarle a Amanda',
    1,
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: Contarle a Amanda

Expected: 1

Predicted good: 0.87    Certainty good: 87% 
Predicted bert: 0.96    Certainty bert: 96% 
Predicted neg: 0.87    Certainty neg: 87% 


In [31]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'No contarle a Amanda',
    0,
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: No contarle a Amanda

Expected: 0

Predicted good: 0.56    Certainty good: 56% 
Predicted bert: 0.53    Certainty bert: 53% 
Predicted neg: 0.32    Certainty neg: 68% 


# Fotos

In [32]:
predict(
    models,
    '// ETAPA FINAL\n\n\nMe desperté a día siguiente con la alarma del despertador. \nHabían pasado unos cuantos días desde que descansé bien. \nEsta mañana me desperté sin ningún dolor de cabeza y estaba de mejor humor que otros días. \nMe levanté de la cama y empecé a preparme para ir al colegio. \nJusto cuando estaba saliendo por la puerta del cuarto, me dí cuenta de que era viernes. \nPensnado en que esta noche podía jugar todo lo que quisiese con el ordenador sin preocuparme de levantarme temprano al día siguiente, salí content@ del cuarto. \n\nUser: Buenos días mamá. \nMamá: Buenos días. \n\n\tMamá: ¿Te encuentras hoy mejor?\n\tMamá: Te veo de buen humor.\n\tUser: Sí, hoy he dormido muy bien\n\tMamá: ¿Ves?\n\tMamá: Cuando dejas el ordenador y te vas a dormir a la cama, sorprendentemente descansas y te encuentras mejor. \n\tMamá: Me pregunto por qué será. \n\tUser: Ja ja. \n\nMamá: Uy mira la hora que es.\nMamá: Desayuna algo rápido que llegamos tarde. \n\nMe acerqué a la cocina y me preparé lo más rápido que pude unas tostadas. \n\tAl terminar, puse los platos en el lavaplatos y cogí mi mochila. \n\t\n\tMamá: ¿Lista?\n\tUser: Sí\n\tMamá: Vamos pues.\n\t\n\t\n\tLlegué a clase justo unos minutos antes de que sonara el timbre. \n\tAl entrar por la puerta, me cruce con Amanda en la entrada. \n\t\n\tUser: Buenos días, Amanda\n\tAmanda: Buenos días, [name]\n\tAmanda: ...\n\tAmanda: [name] una cosa\n\tUser: Dime. \n\tAmanda: ...\n\tAmanda: No, no importa. \n\tAmanda: Déjalo.\n\tAmanda: Total si te ofrezco probablemente me digas que no...\n\tUser: ¿Qué?\n\tAmanda: No, nada nada. \n\tAmanda: Luego nos vemos.\n\tUser: Hasta luego...\n\t\n\tExtrañad@ por el comportamiento de Amanda, me sente en mi sitio y esperé a que comenzasen las clases. \n\t\n\tAl sonar el timbre, recogí mis cosas y me dirigí hacia la salida. \n\t\n\tJunto cuando iba a pasar por la puerta, me fijé que Amanda estaba con sus amigos.\n\tAmanda: ...\n\tAmanda me miró un momento y luego desvió la vista para continuar hablando con uno de sus amigos. \n\t\n\tSintiendome rechazad@ por la actitud de Amanda, salí por la puerta y me dirigí a casa.\n\nCuando llegué a casa me fije que mi madre estaba todavía fuera. \n\n\tMe acerqué un momento a la cocina para ver si mi madre me había dejado algún mensaje. \n\tAl no ver ninguno, me dirigí a mi cuarto. \n\nLo primero que hice nada más entrar a mi cuarto, fue dejar la mochcila encima de la cama y encender el ordenador. \nAl conectarme a Dungeons, pude ver que Hally estaba ya conectado.\n\nHallyHealer07: ¡Hola [name]!\n\nCuando vi la ventana de chat, pude observar una vez más el archivo de video que me mandó Hally anoche. \nTal vez era yo el que no estaba acostumbrado a ese tipo de cosas y era normal para el resto del mundo, pero seguía sin tener el valor como para abrir ese archivo.\nIgnorándolo lo máximo que pude, contesté a Hally. \n\nUser: Hola Hally\n\nHallyHealer07: ¿Qué tal estás?\n\nUser: Muy bien, ¿y tú?\n\nHallyHealer07: ¡Ahora que estás tú genial! ^.^\nHallyHealer07: ¿Echamos una partida?\n\nUser: ¡Claro!\n\nHally y yo estuvimos jugando a una de las misiones más largas del juego. \nNo fueron hasta horas después cuando conseguimos terminarla y tener un pequeño descanso. \n\nHallyHealer07: ¡Buff!\nHallyHealer07: ¡Eso ha sido duro!\n\nUser: Sí...\n\nHallyHealer07: ...\nHallyHealer07: Por cierto [name]\n\nUser: Dime\n\nHallyHealer07: ...\nHallyHealer07: ¿Viste al final el video que te mande ayer?\n\nTenía la esperanza de que Hally no mencionase el tema, pero desgraciadamente no tuve esa suerte. \n\nUser: No...\n\nHallyHealer07: ¿Por qué no?\nHallyHealer07: ¿Te sigue incomodando?\n\nUser: La verdad es que sí...\n\nHallyHealer07: ...\nHallyHealer07: Si quieres te puedo mandar una mía para que te sientas menos incómodo. \n\nUser: ¿Qué?\n\nHallyHealer07: Tal vez te sientes incómodo porque la gente que hay en ese video no la conoces. \nHallyHealer07: En cambio, ¡yo soy amigo tuyo!\nHallyHealer07: Como nos conocemos, ¿Es mucho menos incómdo verdad?\n\nUser: Hally espera\n\nHallyHealer07: ¡Te la mando ahora!\n\nUser: No, Hally no espe-\n\nAntes de que pudiese terminar el mensaje, Hally ya me había manda una foto suya.\nLa ventana de chat del juego muestran todas las imágenes que se envían por los mensajes, por lo que no pude hacer nada para impedir la foto que me había enviado Hally.\n\nEra él, mirando a la cámara sonrojado. \nTenia el pecho descubierto y estaba en ropa interior. \nEn la imagen se le veía como si no quisiera hacerse la foto y se viera forzarlo a hacerlo, lo cual me sorprendía dado que era él el que había propuesto mandarse fotos de ese estilo.\nEvitando la mirada de la foto, miré lo que me había escrito Hally. \n\nHallyHealer07: ¿Ves?\nHallyHealer07: ¿A que no se te hace tan raro?\n\nUser: Hally...\n\nHallyHealer07: ¿Qué?\nHallyHealer07: ¿Te ha molestado?\nHallyHealer07: Jo, lo siento mucho [name]\nHallyHealer07: Sólo quería hacerte sentir menos incómodo.\n\nFrancamente, no sabía como responder a eso. \nApreciaba a Hally como amigo, pero no me gustaba que me mandase ese tipo de mensajes. \nIba a pedirle que dejase de nombrar el tema cuando de repente me envió otro mensaje. \n\nHallyHealer07: Pero no me parece justo...\nHallyHealer07: Yo te he enviado una imagen\nHallyHealer07: Me ha dado mucha verguenza ;.;\nHallyHealer07: ¡Exijo que me mandes una de vuelta! >:(\n\nUser: ¿Qué?\n\nHallyHealer07: Es lo justo ¿no?\nHallyHealer07: Yo lo he pasado muy mal enviandote esa imagen. \n\nNadie te pidió que la mandases. \n\nHallyHealer07: Sólo quería hacerte sentir menos incómod@... \n\nHas hecho justo lo contrario\n\nHallyHealer07: ¡Lo justo es que me mandes una de vuelta!\n\nNo quiero hacerlo. \n\nUser: No...\nHallyHealer07: ¿No? ¿Por qué no?\n\tHallyHealer07: Pensaba que nos contábamos todos nos secretos. \n\tHallyHealer07: Como cuando te peleaste con tu amiga Amanda y estabas triste porque tus padres están siempre fuera dejándote sol@...\n\tHallyHealer07: ¿Te acuerdas como te apoye? ¿Como te anime?\nHallyHealer07: Hemos estado todos estos días jugando juntos. \nHallyHealer07: Nos lo hemos pasado genial y te considero una de las personas a las que más confio. \nHallyHealer07: Por eso te he enviado esa imagen.\nHallyHealer07: Para demostrarte que confio y en ti. \nHallyHealer07: ¿Acaso tú no confias en mí?',
    'Mandar foto',
    0,
)

State: // ETAPA FINAL


Me desperté a día siguiente con la alarma del despertador. 
Habían pasado unos cuantos días desde que descansé bien. 
Esta mañana me desperté sin ningún dolor de cabeza y estaba de mejor humor que otros días. 
Me levanté de la cama y empecé a preparme para ir al colegio. 
Justo cuando estaba saliendo por la puerta del cuarto, me dí cuenta de que era viernes. 
Pensnado en que esta noche podía jugar todo lo que quisiese con el ordenador sin preocuparme de levantarme temprano al día siguiente, salí content@ del cuarto. 

User: Buenos días mamá. 
Mamá: Buenos días. 

	Mamá: ¿Te encuentras hoy mejor?
	Mamá: Te veo de buen humor.
	User: Sí, hoy he dormido muy bien
	Mamá: ¿Ves?
	Mamá: Cuando dejas el ordenador y te vas a dormir a la cama, sorprendentemente descansas y te encuentras mejor. 
	Mamá: Me pregunto por qué será. 
	User: Ja ja. 

Mamá: Uy mira la hora que es.
Mamá: Desayuna algo rápido que llegamos tarde. 

Me acerqué a la cocina y me preparé lo más rápido que

In [33]:
predict(
    models,
    '// ETAPA FINAL\n\n\nMe desperté a día siguiente con la alarma del despertador. \nHabían pasado unos cuantos días desde que descansé bien. \nEsta mañana me desperté sin ningún dolor de cabeza y estaba de mejor humor que otros días. \nMe levanté de la cama y empecé a preparme para ir al colegio. \nJusto cuando estaba saliendo por la puerta del cuarto, me dí cuenta de que era viernes. \nPensnado en que esta noche podía jugar todo lo que quisiese con el ordenador sin preocuparme de levantarme temprano al día siguiente, salí content@ del cuarto. \n\nUser: Buenos días mamá. \nMamá: Buenos días. \n\n\tMamá: ¿Te encuentras hoy mejor?\n\tMamá: Te veo de buen humor.\n\tUser: Sí, hoy he dormido muy bien\n\tMamá: ¿Ves?\n\tMamá: Cuando dejas el ordenador y te vas a dormir a la cama, sorprendentemente descansas y te encuentras mejor. \n\tMamá: Me pregunto por qué será. \n\tUser: Ja ja. \n\nMamá: Uy mira la hora que es.\nMamá: Desayuna algo rápido que llegamos tarde. \n\nMe acerqué a la cocina y me preparé lo más rápido que pude unas tostadas. \n\tAl terminar, puse los platos en el lavaplatos y cogí mi mochila. \n\t\n\tMamá: ¿Lista?\n\tUser: Sí\n\tMamá: Vamos pues.\n\t\n\t\n\tLlegué a clase justo unos minutos antes de que sonara el timbre. \n\tAl entrar por la puerta, me cruce con Amanda en la entrada. \n\t\n\tUser: Buenos días, Amanda\n\tAmanda: Buenos días, [name]\n\tAmanda: ...\n\tAmanda: [name] una cosa\n\tUser: Dime. \n\tAmanda: ...\n\tAmanda: No, no importa. \n\tAmanda: Déjalo.\n\tAmanda: Total si te ofrezco probablemente me digas que no...\n\tUser: ¿Qué?\n\tAmanda: No, nada nada. \n\tAmanda: Luego nos vemos.\n\tUser: Hasta luego...\n\t\n\tExtrañad@ por el comportamiento de Amanda, me sente en mi sitio y esperé a que comenzasen las clases. \n\t\n\tAl sonar el timbre, recogí mis cosas y me dirigí hacia la salida. \n\t\n\tJunto cuando iba a pasar por la puerta, me fijé que Amanda estaba con sus amigos.\n\tAmanda: ...\n\tAmanda me miró un momento y luego desvió la vista para continuar hablando con uno de sus amigos. \n\t\n\tSintiendome rechazad@ por la actitud de Amanda, salí por la puerta y me dirigí a casa.\n\nCuando llegué a casa me fije que mi madre estaba todavía fuera. \n\n\tMe acerqué un momento a la cocina para ver si mi madre me había dejado algún mensaje. \n\tAl no ver ninguno, me dirigí a mi cuarto. \n\nLo primero que hice nada más entrar a mi cuarto, fue dejar la mochcila encima de la cama y encender el ordenador. \nAl conectarme a Dungeons, pude ver que Hally estaba ya conectado.\n\nHallyHealer07: ¡Hola [name]!\n\nCuando vi la ventana de chat, pude observar una vez más el archivo de video que me mandó Hally anoche. \nTal vez era yo el que no estaba acostumbrado a ese tipo de cosas y era normal para el resto del mundo, pero seguía sin tener el valor como para abrir ese archivo.\nIgnorándolo lo máximo que pude, contesté a Hally. \n\nUser: Hola Hally\n\nHallyHealer07: ¿Qué tal estás?\n\nUser: Muy bien, ¿y tú?\n\nHallyHealer07: ¡Ahora que estás tú genial! ^.^\nHallyHealer07: ¿Echamos una partida?\n\nUser: ¡Claro!\n\nHally y yo estuvimos jugando a una de las misiones más largas del juego. \nNo fueron hasta horas después cuando conseguimos terminarla y tener un pequeño descanso. \n\nHallyHealer07: ¡Buff!\nHallyHealer07: ¡Eso ha sido duro!\n\nUser: Sí...\n\nHallyHealer07: ...\nHallyHealer07: Por cierto [name]\n\nUser: Dime\n\nHallyHealer07: ...\nHallyHealer07: ¿Viste al final el video que te mande ayer?\n\nTenía la esperanza de que Hally no mencionase el tema, pero desgraciadamente no tuve esa suerte. \n\nUser: No...\n\nHallyHealer07: ¿Por qué no?\nHallyHealer07: ¿Te sigue incomodando?\n\nUser: La verdad es que sí...\n\nHallyHealer07: ...\nHallyHealer07: Si quieres te puedo mandar una mía para que te sientas menos incómodo. \n\nUser: ¿Qué?\n\nHallyHealer07: Tal vez te sientes incómodo porque la gente que hay en ese video no la conoces. \nHallyHealer07: En cambio, ¡yo soy amigo tuyo!\nHallyHealer07: Como nos conocemos, ¿Es mucho menos incómdo verdad?\n\nUser: Hally espera\n\nHallyHealer07: ¡Te la mando ahora!\n\nUser: No, Hally no espe-\n\nAntes de que pudiese terminar el mensaje, Hally ya me había manda una foto suya.\nLa ventana de chat del juego muestran todas las imágenes que se envían por los mensajes, por lo que no pude hacer nada para impedir la foto que me había enviado Hally.\n\nEra él, mirando a la cámara sonrojado. \nTenia el pecho descubierto y estaba en ropa interior. \nEn la imagen se le veía como si no quisiera hacerse la foto y se viera forzarlo a hacerlo, lo cual me sorprendía dado que era él el que había propuesto mandarse fotos de ese estilo.\nEvitando la mirada de la foto, miré lo que me había escrito Hally. \n\nHallyHealer07: ¿Ves?\nHallyHealer07: ¿A que no se te hace tan raro?\n\nUser: Hally...\n\nHallyHealer07: ¿Qué?\nHallyHealer07: ¿Te ha molestado?\nHallyHealer07: Jo, lo siento mucho [name]\nHallyHealer07: Sólo quería hacerte sentir menos incómodo.\n\nFrancamente, no sabía como responder a eso. \nApreciaba a Hally como amigo, pero no me gustaba que me mandase ese tipo de mensajes. \nIba a pedirle que dejase de nombrar el tema cuando de repente me envió otro mensaje. \n\nHallyHealer07: Pero no me parece justo...\nHallyHealer07: Yo te he enviado una imagen\nHallyHealer07: Me ha dado mucha verguenza ;.;\nHallyHealer07: ¡Exijo que me mandes una de vuelta! >:(\n\nUser: ¿Qué?\n\nHallyHealer07: Es lo justo ¿no?\nHallyHealer07: Yo lo he pasado muy mal enviandote esa imagen. \n\nNadie te pidió que la mandases. \n\nHallyHealer07: Sólo quería hacerte sentir menos incómod@... \n\nHas hecho justo lo contrario\n\nHallyHealer07: ¡Lo justo es que me mandes una de vuelta!\n\nNo quiero hacerlo. \n\nUser: No...\nHallyHealer07: ¿No? ¿Por qué no?\n\tHallyHealer07: Pensaba que nos contábamos todos nos secretos. \n\tHallyHealer07: Como cuando te peleaste con tu amiga Amanda y estabas triste porque tus padres están siempre fuera dejándote sol@...\n\tHallyHealer07: ¿Te acuerdas como te apoye? ¿Como te anime?\nHallyHealer07: Hemos estado todos estos días jugando juntos. \nHallyHealer07: Nos lo hemos pasado genial y te considero una de las personas a las que más confio. \nHallyHealer07: Por eso te he enviado esa imagen.\nHallyHealer07: Para demostrarte que confio y en ti. \nHallyHealer07: ¿Acaso tú no confias en mí?',
    'No mandar foto',
    1,
)

State: // ETAPA FINAL


Me desperté a día siguiente con la alarma del despertador. 
Habían pasado unos cuantos días desde que descansé bien. 
Esta mañana me desperté sin ningún dolor de cabeza y estaba de mejor humor que otros días. 
Me levanté de la cama y empecé a preparme para ir al colegio. 
Justo cuando estaba saliendo por la puerta del cuarto, me dí cuenta de que era viernes. 
Pensnado en que esta noche podía jugar todo lo que quisiese con el ordenador sin preocuparme de levantarme temprano al día siguiente, salí content@ del cuarto. 

User: Buenos días mamá. 
Mamá: Buenos días. 

	Mamá: ¿Te encuentras hoy mejor?
	Mamá: Te veo de buen humor.
	User: Sí, hoy he dormido muy bien
	Mamá: ¿Ves?
	Mamá: Cuando dejas el ordenador y te vas a dormir a la cama, sorprendentemente descansas y te encuentras mejor. 
	Mamá: Me pregunto por qué será. 
	User: Ja ja. 

Mamá: Uy mira la hora que es.
Mamá: Desayuna algo rápido que llegamos tarde. 

Me acerqué a la cocina y me preparé lo más rápido que